## 2021-05-06 Demo notebook querying the BDBSv2 catalog using lsd (procyon version) ##

**2021-05-06:** Demonstrate querying BDBS2 using Mario Juric' Large Survey Database. Two queries are shown: one querying BDBS2 and plotting it, the other using lsd's capability to crossmatch at query time.

Documentation for lsd is a bit incomplete... the most useful link is probably its wiki page, which contains important info and caveats about the way lsd matches: https://github.com/mjuric/lsd/wiki

In [ ]:
%matplotlib inline

## 1. Initial setup - check that we have access to the database ##

In [ ]:
import os
import matplotlib.pylab as plt
import numpy as np

In [ ]:
# where are we running this? (Always handy to know)
print(os.getcwd())

In [ ]:
# astropy tools will be useful
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
os.environ['NWORKERS'] = "4"

# set to 1 to avoid bad message length (but slower processing)
#os.environ['NWORKERS'] = "1"
from lsd import DB
from lsd.bounds import beam, rectangle
from lsd.join_ops import Query

db = None # useful to set this so we can test if it's set later.

In [ ]:
# The tables are subdirectories in the directory indicated by 
# the environment variable LSD_DB. We use the shell to list its
# contents
!ls $LSD_DB

In [ ]:
# On procyon, we have bdbs2 and various external catalogs. Let's demo matching to one of them.

# What columns do we have?
!lsd-admin desc table bdbs2


## Example 1: Query database and plot ##

### 1.1. Build the query ###

In [ ]:
# Decide which columns we want
sql = 'select ra, dec, u, g, r, i, z, y from bdbs2'


In [ ]:
# decide where we're going to look
lCen = 9.9
bCen = -7.55
radDeg = 0.2

# Construct the region we'll use to query this. First example is a circular "beam".
# coordsys 'gal' = galactic, 'equ' = equatorial
# region = beam(lCen, bCen, radDeg, coordsys='gal')

# ... or we could do a rectangle. We specify the lower left and upper right corners:
region = rectangle(lCen-0.4, bCen-0.3, lCen+0.4, bCen+0.3, coordsys='gal')

### 1.2 Connect to the database and execute the query ###

In [ ]:
# Connect to the database
if db is None:
    db = DB(os.environ['LSD_DB'])

In [ ]:
# now we execute the query and convert it to an astropy table
rows = db.query(sql).fetch(bounds=[(region, None)])
tabl = Table(rows.as_ndarray())

In [ ]:
# let's see what columns we have#
print(tabl)

### 1.3 Plot the returned database, do selections, etc. ###

In [ ]:
fig1 = plt.figure(1, figsize=(12,8))
fig1.clf()
ax1 = fig1.add_subplot(111)

rbri = 12.
rfai = 15.

# Let's select some stars that might be interesting
bSho = (tabl['r'] > rbri) & (tabl['r'] < rfai) 

# Our original table does have Galactic coordinates, but we didn't select
# them in the query. So let's calculate Galactic coordinates here.
coo=SkyCoord(tabl['ra']*u.degree, tabl['dec']*u.degree, frame='fk5')
ll = coo.galactic.l.degree
bb = coo.galactic.b.degree

# set a radius array that scales with the color value
rval = tabl['r'][bSho]
sval = 10 + 49*( (np.max(rval) - rval) / (np.max(rval)-np.min(rval)) )

dum = ax1.scatter(ll[bSho], bb[bSho], c=tabl['r'][bSho], edgecolor='None', cmap='viridis', s=sval, \
                 vmin=rbri, vmax=rfai)

cbar = fig1.colorbar(dum, ax=ax1, label='r (BDBS)')


ax1.invert_xaxis()
ax1.set_xlabel('l (degree)')
ax1.set_ylabel('b (degree)')

#ax1.set_xlabel(r'$\alpha$ (BDBS)')
#ax1.set_ylabel(r'$\delta$ (BDBS)')
ax1.grid(which='both', visible=True)

ax1.set_title('Query: %i selected' % (np.sum(bSho)))

## Example: crossmatch with another database at query time ##

lsd's "killer feature" is its ability to do cross-matches at query time. Here's an example. Suppose we want to cross-match bdbs2 against VVV at query time:

### 2.1. List the columns in the two tables ###

In [ ]:
# What columns do we hvae in vvv?
!lsd-admin desc table vvvPhotFull

In [ ]:
# columns in the original bdbs2
!lsd-admin desc table bdbs2

### 2.2 Construct the query including crossmatching ###

We illustrate syntax that crossmatches the two tables and includes data from the two tables in the output.

We will query BDBS2 and match it to the VVV aperture photometry catalog

In [ ]:
# I find it easier to read if I build the query line by line. We'll print it at the end
# to see what we created.

# The columns we want from each table...
squery = 'select ra, dec, vvvPhotFull.RA2000 as vvv_ra, vvvPhotFull.DEC2000 as vvv_dec'
squery = '%s, u, g, r, i, z, y' % (squery)

# Now let's add some VVV photometry
squery = '%s, vvvPhotFull.ZAPERMAG3 as vvv_z, vvvPhotFull.YAPERMAG3 as vvv_y' % (squery)
squery = '%s, vvvPhotFull.JAPERMAG3 as vvv_j, vvvPhotFull.HAPERMAG3 as vvv_h' % (squery)
squery = '%s, vvvPhotFull.KSAPERMAG3 as vvv_ks' % (squery)

# The distance in arcsec between gaia and BDBS2:
squery = '%s, vvvPhotFull._DIST*3600 as dist from bdbs2' % (squery)

# Instruct lsd to match bdbs to vvv
squery = '%s, vvvPhotFull(matchedto=bdbs2)' % (squery)

In [ ]:
# let's see what this query looks like
print(squery)

In [ ]:
# We already defined the region upstream in this notebook... Uncomment the following to change it

## decide where we're going to look
#lCen = 9.9
#bCen = -7.55
#radDeg = 0.2

## Construct the region we'll use to query this. First example is a circular "beam".
## coordsys 'gal' = galactic, 'equ' = equatorial
#region = beam(lCen, bCen, radDeg, coordsys='gal')

## ... or we could do a rectangle. We specify the lower left and upper right corners:
#region = rectangle(lCen-0.4, bCen-0.3, lCen+0.4, bCen+0.3, coordsys='gal')

In [ ]:
# Ensure we are connected to the database, and query the catalog
if db is None:
    db = DB(os.environ['LSD_DB'])
    
rows2 = db.query(squery).fetch(bounds=[(region, None)])
tabl2 = Table(rows2.as_ndarray())

In [ ]:
# Let's see what we just retrieved
print(tabl2)

### 2.3. Plot the results of this query ###

In [ ]:
fig2 = plt.figure(2, figsize=(12,8))
fig2.clf()
ax2 = fig2.add_subplot(111)

rbri = 12.
rfai = 15.

# Let's select some stars that might be interesting
bBri = (tabl2['r'] > rbri) & (tabl2['r'] < rfai) 

# select by distance
bSel = tabl2['dist'] < 0.2

bSho = (bBri) & (bSel)


# Our original table does have Galactic coordinates, but we didn't select
# them in the query. So let's calculate Galactic coordinates here.
coo=SkyCoord(tabl2['ra']*u.degree, tabl2['dec']*u.degree, frame='fk5')
ll2 = coo.galactic.l.degree
bb2 = coo.galactic.b.degree

# set a radius array that scales with the color value
rval = tabl2['r'][bSho]
sval = 10 + 49*( (np.max(rval) - rval) / (np.max(rval)-np.min(rval)) )

dum2 = ax2.scatter(ll2[bSho], bb2[bSho], c=tabl2['r'][bSho], edgecolor='None', cmap='viridis', s=sval, \
                 vmin=rbri, vmax=rfai)

cbar2 = fig2.colorbar(dum2, ax=ax2, label='r (BDBS)')


ax2.invert_xaxis()
ax2.set_xlabel('l (degree)')
ax2.set_ylabel('b (degree)')

#ax2.set_xlabel(r'$\alpha$ (BDBS)')
#ax2.set_ylabel(r'$\delta$ (BDBS)')
ax2.grid(which='both', visible=True)

ax2.set_title('Cross-matched at query: %i selected' % (np.sum(bSho)))

### 2.4. Let's try comparing BDBS2 and VVV z-band photometry ###

In [ ]:
bGood = (np.isfinite(tabl2['vvv_z'])) & (tabl2['z'] > 0) & (tabl2['dist'] < 0.2)
bGood = (bGood) & (tabl2['vvv_z'] < 90) & (tabl2['z'] < 90)

# how does this look?
fig2 = plt.figure(2, figsize=(12,5))
fig2.clf()
ax2 = fig2.add_subplot(111)
blah = ax2.scatter(tabl2['z'][bGood], \
                tabl2['z'][bGood] - tabl2['vvv_z'][bGood], \
                c=tabl2['dist'][bGood], edgecolor='None', alpha=0.2, \
               cmap='plasma_r', s=5, zorder=5)
ax2.set_xlim(12,22)
ax2.set_ylim(-3,8)
ax2.set_xlabel('z (BDBS2)', fontsize=14)
ax2.set_ylabel('z(BDBS) - z(VVV)', fontsize=14)
cm = fig2.colorbar(blah, ax=ax2, label='Separation (arcsec)')
ax2.set_title('z-band comparison: BDBS2 to VVV aperture mag z', fontsize=14)
ax2.grid(which='both', zorder=1, visible=True, alpha=0.5)


### 2.5. Any examples with measurements at ugrizJHKs? ###

In [ ]:
# let's loop through the filters
bSel = tabl2['dist'] < 0.3 # arcsec
for sCol in ['u', 'g', 'r', 'i', 'z', 'y', 'vvv_j', 'vvv_h', 'vvv_ks']:
    
    # find the good measurements in this filter
    bThis = (tabl2[sCol] < 90) & (np.isfinite(tabl2[sCol])) & (tabl2[sCol] > 0)
    
    bSel = (bSel) & (bThis)
    
    # monitor how the selection dwindles as we add more constraints
    print("Selecting further on %s: %i" % (sCol, np.sum(bSel)))

In [ ]:
# For convenience, let's create a new view of the entire table to avoid having
# to type all those booleans
tSel = tabl2[bSel]

# Again, I forgot to bring the Galactic coordinates in. So let's compute them.
coo2 = SkyCoord(tSel['ra']*u.degree, tSel['dec']*u.degree, frame='fk5')
l2 = coo2.galactic.l.degree
b2 = coo2.galactic.b.degree

# where is our cluster centered? (Give this a new pair
# of variables so that we can move it if needed)
lc = 9.89
bc = -7.54

# Generate an approximate distance array from the field center
radii = np.sqrt((l2 - lCen)**2 + (b2-bCen)**2)

# try a wide color CMD
fig3 = plt.figure(3, figsize=(14,8))
fig3.clf()
ax3 = fig3.add_subplot(121)

dum2 = ax3.scatter(tSel['u'] - tSel['vvv_ks'], tSel['i'], alpha=0.2, \
                   c=radii, \
                   edgecolor='None', s=8, \
                  cmap='plasma', zorder=5)

ax3.set_xlabel(r'u (BDBS2) - K$_s$ (VVV)', fontsize=14)
ax3.set_ylabel(r'i (BDBS2)', fontsize=14)

# ax3.grid(which='both', visible=True, alpha=0.5, zorder=1)

ax3.set_xlim(0,9)
ax3.set_ylim(13,20)
ax3.invert_yaxis()
ax3.set_title('Color coded by separation from cluster center')

cm3 = fig3.colorbar(dum2, ax=ax3, label='distance (arcsec)')

# let's show the coords of our matching objects

# Take a reasonably bright sample to avoid cluttering the plot
bBri = tSel['i'] < 18

axhoriz = ax3.axhline(18, ls='--', zorder=10, color='k')

ax4 = fig3.add_subplot(222)
dum4 = ax4.scatter(l2[bBri], b2[bBri], c=radii[bBri], cmap='plasma', \
                  edgecolor='None', s=2, alpha=0.5)
ax4.set_xlabel('l (deg)', fontsize=14)
ax4.set_ylabel('b (deg)', fontsize=14)
ax4.invert_xaxis()
ax4.set_title('Cluster center indicated')
cm4 = fig3.colorbar(dum4, ax=ax4, label='distance (arcsec)')


blah2 = ax4.scatter(lc, bc, marker='*', c='c', edgecolor='c', s=100, zorder=15, \
                   label='cluster center')
leg4=ax4.legend(scatterpoints=1)

# How do the positions compare?
ax5 = fig3.add_subplot(224)
dra = (tSel['ra'] - tSel['vvv_ra']) * np.cos(np.radians(tSel['dec']))
dde = tSel['dec'] - tSel['vvv_dec']
dum5 = ax5.scatter(dra[bBri]*3600., \
                  dde[bBri]*3600., \
                  c=radii[bBri], \
                   #vmin=15., vmax=18., \
                   s=2, \
                   alpha=0.20, \
                   cmap='plasma', \
                  edgecolor='None')
ax5.set_ylim(-0.25, 0.25)
ax5.set_xlim(-0.3, 0.3)
ax5.set_xlabel(r'$\Delta \alpha \cos \delta$ (")', fontsize=16)
ax5.set_ylabel(r'$\Delta \delta$ (")', fontsize=16)
ax5.set_title('BDBS - VVV separations (i < 18)')
cm5 = fig3.colorbar(dum5, ax=ax5, label='distance (arcsec)')

# let's pick a random BHB object and print out its apparent magnitudes
bBHB = (tSel['u'] - tSel['vvv_ks'] < 2) & (tSel['i'] < 15.5)
lRows = np.arange(len(tSel))
lBHB = lRows[bBHB][3]
for sCol in ['u', 'g', 'r', 'i', 'z', 'vvv_z', 'y', 'vvv_y', 'vvv_j', 'vvv_h', 'vvv_ks']:
    print("%s: %.2f" % (sCol, tSel[sCol][lBHB]))
    
# add grids
for ax in [ax3, ax4, ax5]:
    ax.grid(which='both', visible=True, alpha=0.5, zorder=1)
    
fig3.subplots_adjust(wspace=0.3, hspace=0.3)

# save the figure as rasterized png
fig3.savefig('tmp_cmd.png', rasterized=True)


In [ ]:
# we can write the selected table to disk
tSel.write('tmp_query_subset.fits.gz', overwrite=True)

In [ ]:
!ls -rtlh tmp_*.*